In [ ]:
!pip install torch
!pip install torchvision

In [16]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

from tqdm import tqdm

In [4]:
train_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

100%|██████████| 26.4M/26.4M [00:00<00:00, 51.2MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 1.15MB/s]
100%|██████████| 4.42M/4.42M [00:00<00:00, 20.4MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 13.2MB/s]


In [7]:
batch_size = 64

train_dataloader = DataLoader(train_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [11]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"

In [18]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

nn_model = Model().to(device)

In [19]:
loss_func = nn.CrossEntropyLoss()
optim = torch.optim.SGD(nn_model.parameters(), lr=1e-3)

In [20]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)

        pred = model(x)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(x)
            print(f"Loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [21]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test loss: {test_loss:8f}, Accuracy: {100*correct}")

In [22]:
epochs = 5
for e in tqdm(range(epochs)):
    train(train_dataloader, nn_model, loss_func, optim)
    test(test_dataloader, nn_model, loss_func)

  0%|          | 0/5 [00:00<?, ?it/s]

Loss: 2.297118 [   64/60000]
Loss: 2.280867 [ 6464/60000]
Loss: 2.260597 [12864/60000]
Loss: 2.259923 [19264/60000]
Loss: 2.245459 [25664/60000]
Loss: 2.209290 [32064/60000]
Loss: 2.223743 [38464/60000]
Loss: 2.187803 [44864/60000]
Loss: 2.180397 [51264/60000]
Loss: 2.156875 [57664/60000]


 20%|██        | 1/5 [00:07<00:28,  7.13s/it]

Test loss: 2.145408, Accuracy: 52.94
Loss: 2.154281 [   64/60000]
Loss: 2.144095 [ 6464/60000]
Loss: 2.086032 [12864/60000]
Loss: 2.104253 [19264/60000]
Loss: 2.056720 [25664/60000]
Loss: 1.994230 [32064/60000]
Loss: 2.025739 [38464/60000]
Loss: 1.945564 [44864/60000]
Loss: 1.946224 [51264/60000]
Loss: 1.879334 [57664/60000]


 40%|████      | 2/5 [00:14<00:21,  7.29s/it]

...

Test loss: 1.507570, Accuracy: 59.84
Loss: 1.578828 [   64/60000]
Loss: 1.543256 [ 6464/60000]
Loss: 1.392362 [12864/60000]
Loss: 1.462257 [19264/60000]
Loss: 1.346919 [25664/60000]
Loss: 1.348566 [32064/60000]
Loss: 1.363443 [38464/60000]
Loss: 1.287381 [44864/60000]
Loss: 1.328349 [51264/60000]
Loss: 1.225187 [57664/60000]


 80%|████████  | 4/5 [00:29<00:07,  7.45s/it]

Test loss: 1.250329, Accuracy: 62.82
Loss: 1.331267 [   64/60000]
Loss: 1.310716 [ 6464/60000]
Loss: 1.146997 [12864/60000]
Loss: 1.248446 [19264/60000]
Loss: 1.126672 [25664/60000]
Loss: 1.156549 [32064/60000]
Loss: 1.179003 [38464/60000]
Loss: 1.111658 [44864/60000]
Loss: 1.160649 [51264/60000]
Loss: 1.070227 [57664/60000]


100%|██████████| 5/5 [00:37<00:00,  7.44s/it]

Test loss: 1.090785, Accuracy: 64.31
